In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np
import cv2
from google.colab import drive
import os

In [ ]:
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive', force_remount=True)

# Set the base directory for your dataset
base_dir = '/content/drive/MyDrive/Soil_Images'

Mounted at /content/drive


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt

# Set image size and batch size
img_height, img_width = 150, 150
batch_size = 32

# Data augmentation and preprocessing
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2)  # 80% training, 20% validation

# Train set
train_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training')

# Validation set
validation_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation')

# Get class labels
class_labels = list(train_generator.class_indices.keys())
print(f"Class Labels: {class_labels}")

Found 134 images belonging to 5 classes.
Found 32 images belonging to 5 classes.
Class Labels: ['Black_Soil', 'Cinder_Soil', 'Laterite_Soil', 'Peat_Soil', 'Yellow_Soil']


In [ ]:
#Defining CNN model
model = Sequential([
    Input(shape=(img_height, img_width, 3)),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(class_labels), activation='softmax')
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
#Traininig model
epochs = 10
history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator
)

Epoch 1/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 10s 2s/step - accuracy: 0.9048 - loss: 0.3311 - val_accuracy: 0.8750 - val_loss: 0.3245
Epoch 2/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 10s 1s/step - accuracy: 0.8653 - loss: 0.3711 - val_accuracy: 0.7812 - val_loss: 0.3795
Epoch 3/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 12s 2s/step - accuracy: 0.8589 - loss: 0.3506 - val_accuracy: 0.8750 - val_loss: 0.2759
Epoch 4/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 10s 2s/step - accuracy: 0.8428 - loss: 0.4726 - val_accuracy: 0.7812 - val_loss: 0.4444
Epoch 5/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 9s 1s/step - accuracy: 0.7991 - loss: 0.4300 - val_accuracy: 0.8438 - val_loss: 0.4518
Epoch 6/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 11s 2s/step - accuracy: 0.8481 - loss: 0.4483 - val_accuracy: 0.7188 - val_loss: 0.6302
Epoch 7/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 9s 2s/step - accuracy: 0.7680 - loss: 0.5526 - val_accuracy: 0.8438 - val_loss: 0.5289
Epoch 8/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 10s 2s/step - accuracy: 0.8223 - loss: 0.5036 - val_accuracy: 0.7812 - val_loss: 0.5929
Epoch 9/10

In [ ]:
#evaluating model
loss, accuracy = model.evaluate(validation_generator)
print(f"Validation Accuracy: {accuracy:.2f}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.8438 - loss: 0.6437
Validation Accuracy: 0.84


In [ ]:
model_save_path = '/content/drive/MyDrive/soil_model.h5'
model.save(model_save_path)
print(f"Model saved to {model_save_path}")  #model saving

Model saved to /content/drive/MyDrive/soil_model.h5


In [ ]:
#making prediction
from tensorflow.keras.preprocessing import image
import numpy as np
from google.colab import files

def predict_soil_type(img_path):
    # Load the image and preprocess
    img = image.load_img(img_path, target_size=(img_height, img_width))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0  # Normalize

    # Predict using the model
    predictions = model.predict(img_array)
    predicted_class_index = np.argmax(predictions[0])
    predicted_class = class_labels[predicted_class_index]
    confidence_score = predictions[0][predicted_class_index]

    return predicted_class, confidence_score

# Example usage
uploaded = files.upload()
for img_name in uploaded.keys():
    soil_type, confidence = predict_soil_type(img_name)
    print(f"Predicted Soil Type: {soil_type}, with Confidence: {confidence:.2f}")